In [37]:
import sys

sys.path.append("../../src/stong_core_cython")

In [38]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
import concurrent.futures
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)

from atom import *
from hartree_fock import *
from basis_data import *

In [39]:
df = pd.read_csv("../make_base/sto_3g.csv")

In [40]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 3, 0)}
    hartreefock = HartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [41]:
elements = [
    "H",
    "He",
    "Li",
    "Be",
]
# elements = ["H"]
zetas = {}
results = {}
for i, element in enumerate(elements):
    print(element)
    if i == 0:
        result = minimize(get_energy, [5, 2.69, 1], args=(element))
    else:
        result = minimize(get_energy, result.x, args=(element))
    # result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    results[element] = result
    zetas[element] = result.x
    print(zetas[element])

H
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, p

In [42]:
results

{'H':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -0.4987485688712988
         x: [-1.797e+00  2.322e+00  1.510e+00]
       nit: 30
       jac: [-4.172e-07  1.267e-06 -1.155e-06]
  hess_inv: [[ 1.427e+02 -7.637e+01 -3.864e+01]
             [-7.637e+01  1.892e+02  6.348e+01]
             [-3.864e+01  6.348e+01  2.849e+01]]
      nfev: 140
      njev: 35,
 'He':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -2.850110070174953
         x: [-3.104e+00  3.672e+00  2.325e+00]
       nit: 23
       jac: [-8.941e-08  1.788e-07 -6.557e-07]
  hess_inv: [[ 7.660e+01 -2.275e+01 -1.371e+01]
             [-2.275e+01  9.942e+01  3.259e+01]
             [-1.371e+01  3.259e+01  1.436e+01]]
      nfev: 108
      njev: 27,
 'Li':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -7.3761092841262865
         x: [-3.787e+00  2.985e+00  9.442e-01]
       nit: 17
       jac:

In [43]:
df_zetas = pd.DataFrame(zetas)
df_zetas.to_csv("zetas.csv", index=False)